In [1]:
import numpy as np

### Step 1: Simulate Textual Maintenance Reports and Operational Data
First, we simulate maintenance reports and sensor data. For simplicity, these examples will be basic but aim to capture the essence of real data:

In [2]:
# Simulate some maintenance reports with keywords indicating potential issues
maintenance_reports = [
    "Routine check. No issues detected.",
    "Reported overheating in the engine compartment.",
    "Vibration levels higher than normal. Inspection recommended.",
    "Leak detected in the cooling system.",
    "Routine check. Minor wear on belts.",
    "Unusual noise heard from the gearbox. Further analysis required."
]

# Simulate some operational sensor data (e.g., temperature readings over time for each report)
sensor_data_temp = [
    [21, 22, 20, 21, 20],  # Normal temperature readings
    [30, 31, 29, 32, 33],  # Overheating
    [22, 23, 21, 22, 24],  # Normal with slight variation
    [25, 26, 27, 28, 29],  # Gradual increase indicating a leak or failure
    [19, 20, 21, 20, 19],  # Normal wear conditions
    [22, 30, 23, 25, 24]   # Unusual spike indicating possible issues
]

### Step 2: Feature Engineering
Let's extract features from both the textual maintenance reports using an LLM for semantic understanding and the operational sensor data for statistical insights.

#### Textual Data Feature Engineering
Using a simple bag-of-words approach for keywords indicating potential issues, transforming reports into features:

In [14]:
from transformers import BertTokenizer, BertModel
import torch

# Initialize BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

def generate_embeddings(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    outputs = model(**inputs)
    # Use the pooled output as it provides a fixed size embedding for variable length text
    embeddings = outputs.pooler_output
    return embeddings

# Generate embeddings for each report
report_embeddings = torch.stack([generate_embeddings(report) for report in maintenance_reports])


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

#### Operational Data Feature Engineering
For the operational sensor data, we compute statistical features such as mean and standard deviation to capture the behavior over time:

In [4]:
op_features = []

for readings in sensor_data_temp:
    mean_temp = np.mean(readings)
    std_temp = np.std(readings)
    max_temp = np.max(readings)
    min_temp = np.min(readings)
    op_features.append([mean_temp, std_temp, max_temp, min_temp])

op_features = np.array(op_features)


### Step 3: Combine Features and Prepare Dataset
Combining the engineered features from textual and operational data:

In [19]:
# Assuming operational features are stored in `op_features` as a NumPy array
op_features_tensor = torch.tensor(op_features, dtype=torch.float32)  # Convert operational features to a tensor

# Remove the singleton dimension from report_embeddings
report_embeddings_squeezed = report_embeddings.squeeze(1)

# Concatenate text embeddings with operational features
combined_features = torch.cat((report_embeddings_squeezed, op_features_tensor), dim=1)

For labels, let's assume **1** indicates a report followed by a failure within the next period, and **0** indicates no failure. In practice, this would be derived from historical data:

In [21]:
# Simulating labels based on the described issues (1 for potential failure, 0 for no issue)
labels = np.array([0, 1, 1, 1, 0, 1])  # Placeholder for actual failure occurrence data

### Splitting the Dataset and Creating DataLoaders


In [22]:
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset, random_split
import torch

# Convert to PyTorch tensors
features_tensor = torch.tensor(combined_features, dtype=torch.float32)
labels_tensor = torch.tensor(labels, dtype=torch.float32)

# Splitting the dataset
X_train, X_test, y_train, y_test = train_test_split(features_tensor, labels_tensor, test_size=0.2, random_state=42)

# Creating TensorDatasets
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

# Creating DataLoaders
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

<ipython-input-22-befc020a327e>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  features_tensor = torch.tensor(combined_features, dtype=torch.float32)


### Defining the LSTM Model
Here's an LSTM model defined using PyTorch, tailored for binary classification (predicting failure: yes or no):

In [23]:
import torch.nn as nn
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim=1, num_layers=1):
        super(LSTMModel, self).__init__()
        self.hidden_dim = hidden_dim
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True)
        self.linear = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        lstm_out, _ = self.lstm(x)
        lstm_out = lstm_out[:, -1, :]
        out = self.linear(lstm_out)
        return out

model = LSTMModel(input_dim=combined_features.shape[1], hidden_dim=50)

### Training the Model
Now, we implement the training loop including calculating loss and updating model parameters:

In [24]:
import torch.optim as optim

criterion = nn.BCEWithLogitsLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

def train_model(model, train_loader, criterion, optimizer, epochs=20):
    model.train()
    for epoch in range(epochs):
        running_loss = 0.0
        for inputs, labels in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs.unsqueeze(1))  # Add a dimension [batch, time_step, features]
            loss = criterion(outputs.squeeze(), labels)  # Squeeze output to match label shape
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch {epoch+1}, Loss: {running_loss/len(train_loader)}")

train_model(model, train_loader, criterion, optimizer)


Epoch 1, Loss: 0.709526002407074
Epoch 2, Loss: 0.6526556015014648
Epoch 3, Loss: 0.6121581792831421
Epoch 4, Loss: 0.5796563029289246
Epoch 5, Loss: 0.5561224818229675
Epoch 6, Loss: 0.540666937828064
Epoch 7, Loss: 0.5273427367210388
Epoch 8, Loss: 0.512274980545044
Epoch 9, Loss: 0.496063232421875
Epoch 10, Loss: 0.4820007085800171
Epoch 11, Loss: 0.4693059027194977
Epoch 12, Loss: 0.4535234570503235
Epoch 13, Loss: 0.437250554561615
Epoch 14, Loss: 0.4248446226119995
Epoch 15, Loss: 0.41298580169677734
Epoch 16, Loss: 0.4000466763973236
Epoch 17, Loss: 0.38636136054992676
Epoch 18, Loss: 0.37255609035491943
Epoch 19, Loss: 0.3601023554801941
Epoch 20, Loss: 0.3483319878578186


### Evaluating the Model
To evaluate the model's performance on unseen data, you can use the test dataset:

In [27]:
def evaluate_model(model, test_loader, criterion):
    model.eval()
    total_loss = 0
    correct_predictions = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            outputs = model(inputs.unsqueeze(1))
            loss = criterion(outputs.squeeze(), labels)
            total_loss += loss.item()
            predicted = torch.sigmoid(outputs).round()  # Applying sigmoid to get [0,1] range and rounding off
            correct_predictions += (predicted.squeeze() == labels).sum().item()
    avg_loss = total_loss / len(test_loader.dataset)
    accuracy = correct_predictions / len(test_loader.dataset)
    print(f"Test Loss: {avg_loss:.4f}, Accuracy: {accuracy:.4f}")

evaluate_model(model, test_loader, criterion)

Test Loss: 0.2360, Accuracy: 0.5000


## Generate test predictions

In [29]:
model.eval()  # Set the model to evaluation mode
predicted = torch.tensor([])  # Initialize an empty tensor to store predictions

with torch.no_grad():  # No need to track gradients during prediction
    for inputs, _ in test_loader:  # We only need the inputs from the test_loader
        inputs = inputs.unsqueeze(1)  # Add an extra dimension for LSTM
        outputs = model(inputs)  # Get the model's predictions (logits)
        probs = torch.sigmoid(outputs)  # Apply sigmoid to convert logits to probabilities
        batch_predicted = probs.round()  # Round probabilities to get binary predictions
        predicted = torch.cat((predicted, batch_predicted), dim=0)  # Aggregate predictions

# Now you have `predicted` filled with your model's predictions


FOr this case, Precision, Recall, and F1 Score are much more important than the regular accuracy metric and returning positive cases (failure, etc) is much more important.

### Precision, Recall, and F1 Score

Precision measures the accuracy of positive predictions. It’s the ratio of true positive predictions to the total number of positive predictions (including false positives).

Recall (Sensitivity) measures the ability of the model to detect all actual positives. It’s the ratio of true positive predictions to the total actual positives (including false negatives).

F1 Score provides a balance between precision and recall, offering a single metric to assess performance when both are important. It's the harmonic mean of precision and recall.

### Calculating Metrics in PyTorch
After evaluating your model, you can calculate these metrics as follows:

In [30]:
from sklearn.metrics import precision_score, recall_score, f1_score

# Assuming `y_test` and `predicted` are available from your model's test phase
# y_test: actual labels
# predicted: model's predictions

# Convert tensors to NumPy arrays for compatibility with scikit-learn metrics
y_true_np = y_test.numpy()
predicted_np = predicted.numpy().round()  # Assuming your model outputs probabilities that need rounding

precision = precision_score(y_true_np, predicted_np)
recall = recall_score(y_true_np, predicted_np)
f1 = f1_score(y_true_np, predicted_np)

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"F1 Score: {f1:.4f}")


Precision: 0.5000
Recall: 1.0000
F1 Score: 0.6667
